### ELN WRITEUP COMPARISON BETWEEN DM INSTANCES:

- run each 'cell' one by one; once you get to [Cell 3](#cell-3):
- simply select the CRO protocol name and click 'Submit' button
    - it will read the list of experiment ids from the text file
- afterwards when you reach [Cell 4](#cell-4) an html render of those specific experiment ids will be executed
- check the valid/invalid checkboxes as you are QC'ing the writeup data
- when finished, run [Cell 5](#cell-5) and a blue button will appear; click this to save the checkboxes
- lastly, execute [Cell 6](#cell-6) in order to view the saved experiment ids and their `validity` status; you can download the `.csv` file

## Cell 1

In [ ]:
from os import getenv
from dotenv import load_dotenv
load_dotenv()
DB_USER = getenv("DB_USER")
DB_PASS = getenv("DB_PASS")
DB_HOST = getenv("DB_HOST")
DB_PORT = getenv("DB_PORT")
DB_NAME = getenv("DB_NAME")

## Cell 2

In [ ]:
%run compare_eln_writeup_scrape.ipynb

## Cell 3

In [ ]:
display(cro_dropdown, cro_button, output)

##### Only run [Cell 4](#cell-4) after executing each cell above and selecting the cro protocol then click 'Submit'

## Cell 4

In [ ]:
idx = 0
dd_value = cro_dropdown.value
print ('='*15, f'{dd_value}', '='*15)
html_output = ""
exp_ids = []
with open(os.path.join(dir_path, file_name_prefix.format(dd_value)), 'r') as file:
    for line in file:
        exp_ids.extend(line.strip().split(","))
        
filtered_df = df[df['exp_id'].isin(exp_ids)].copy()
filtered_df['write_up'] = filtered_df.apply(color_code_writeup, axis=1) 

for exp_id, group in filtered_df.groupby('exp_id'):
    idx += 1
    html_output += render_html_group(idx, group)
html_output += js_code
display(HTML(html_output))

##### Run [Cell 5](#cell-5) and click the 'Update Valid Exp Ids' blue button

## Cell 5

In [ ]:
display(Javascript(save_btn))

##### Run [Cell 6](#cell-6) and download .csv file via the button

## Cell 6

In [ ]:
df_val = pd.read_sql(sql_query.format(f"validity_{user_name_uscore}") , engine)
df_val.sort_values(by='exp_id', ascending=False, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_val.to_csv(csv_buffer, index=False)
csv_content = csv_buffer.getvalue()
csv_content = urllib.parse.quote(csv_content)
download_link = download_btn_template.format(csv_content=csv_content, user_name=user_name_uscore)
display(HTML(download_link))
display(HTML(df_val.to_html()))